#### 概觀測試

In [ ]:
# # overview test
# import time
# headers={'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Mobile Safari/537.36',
#          'accept-language': 'zh_TW'}
# proxy=['60.251.62.27','47.243.196.231','54.37.160.93','199.19.226.12','160.16.233.90','36.234.46.7','23.254.250.94']
# domain='https://www.klook.com'
# start=4
# for s in range(1,3):
#                 tmp=requests.get(domain+'/v1/experiencesrv/category/activity?frontend_id_list=19&size=24&start='+str(s),
#                                 headers=headers,
#                                 proxies={'http':'http://'+random.choice(proxy)})
#                 time.sleep(5)


In [ ]:
# def test(l,t):
#     l=l.split("/")[-2].split("-")[0]
#     if("|" in t):
#         t=int(t.split("|")[0].replace('則評價',""))
#     else:
#         t=0
#     if(t%10==0.0):
#         p=int(t/10)
#     else:
#         p=int(t/10)+1
#     print(l,t,p)
# activity=tmp.json()['result']['activities']
# for a in (activity):
#     name=a['title']
#     link=a['deep_link']
#     talkcnt=a['review_hint']
#     star=a['review_star']
#     test(link,talkcnt)
#     print(name,link,talkcnt,star)

In [ ]:
# tmpC=requests.get('https://www.klook.com/v1/usrcsrv/activities/51850/reviews?page=2&limit=10&star_num=&lang=&sort_type=0&only_image=false&translate=1',
#                 headers=headers,
#                 proxies={'http':'http://'+random.choice(proxy)})
# item=tmpC.json()['result']['item']
# for a in (item):
#     comment=a['content']
#     rank=a['rating']/20
#     print(comment,rank)

In [ ]:
#tmp1

# 爬取&儲存

In [ ]:
def request(key,num,tag):
    
    try:
        time.sleep(3)
        if key=='product': 
            for s in range(1,num+1):
                url=requests.get(domain+'/v1/experiencesrv/category/activity?frontend_id_list=19&size=24&start='+str(s),
                                headers=headers,
                                proxies={'http':'http://'+random.choice(proxy)})
                time.sleep(5)
                if url.status_code==200:
                    getList(url)

        elif key=='comment':
            for s in range(1,num+1):
                url=requests.get(domain+'/v1/usrcsrv/activities/'+tag+'/reviews?page='+str(s)+'&limit=10&star_num=&lang=&sort_type=0&only_image=false&translate=1',
                        headers=headers,
                        proxies={'http':'http://'+random.choice(proxy)})
                time.sleep(5)
                if url.status_code==200:
                    getComment(url,tag)
        else:
            print('try again')

        saveData("",2)
    except Exception as e:
        print(e)
#soup=BeautifulSoup(url.text,'html.parser')
#soup.find_all('h3',class_='activity-card-title')

In [ ]:
def parse(link,cnt):

    actID=link.split("/")[-2].split("-")[0]
    if("|" in cnt):
        cnt=int(cnt.split("|")[0].replace('則評價',""))
    else:
        cnt=0
    if(cnt%10==0.0):
        page=int(cnt/10)
    else:
        page=int(cnt/10)+1
   # print(actID,cnt,page)
    request('comment',page,actID)
    

In [ ]:
def getList(url):
    activity=url.json()['result']['activities']
    global times
    for a in activity:
        name=a['title']
        link=a['deep_link']
        talkcnt=a['review_hint']
        star=a['review_star']
        pid=times
        saveData([pid,name,link,talkcnt,star],0)
        parse(link,talkcnt)
        times+=1


        

In [ ]:
def getComment(url,tag):
    try:
        item=url.json()['result']['item']
        for a in (item):
            comment=a['content']
            rank=a['rating']/20
            cid=tag
            saveData([cid,comment,rank],1)
    except:
        pass

In [ ]:
def saveData(arg,key):
    CID,PID=0.0,0.0
    product,ref,cnt,score,word,rate="","",0.0,0.0,"",0.0
    global data,tmp1,tmp2,ALLdic2,alldict2
    if key==0:
        CID=arg[0]
        product=arg[1]
        ref=arg[2]
        cnt=arg[3]
        score=arg[4]
        tmp1=dict(CID=CID,product=product,ref=ref,cnt=cnt,score=score)
        show(dict(CID=CID,product=product,ref=ref,cnt=cnt,score=score))
    elif key==1:
        PID=arg[0]
        word=arg[1]
        rate=arg[2]
        tmp2=dict(PID=PID,word=word,rate=rate)
        alldict2.append(tmp2)
        ALLdic2={'opinion':alldict2}
        show(dict(PID=PID,word=word,rate=rate))
    elif key==2:
        data.append([tmp1,ALLdic2])
        print("-----nextList----")
        toDB(tmp1,ALLdic2)
        alldict2.clear()
    else:
        print('no value')
    
    
        

    

In [ ]:
def show(info):
    print(info)

In [ ]:
#pip install pymongo

In [ ]:
import pymongo
client=pymongo.MongoClient('localhost',27017)
testdb=client['TEST']
tmp=testdb['TMP']
def toDB(dic1,dic2):
    try:
        global yid
        dic1.update(dic2)
        yid.update(dic1)
        if yid in tmp.find():
            tmp.update_one({},{"$set":yid})
        else:
            tmp.insert_one(yid)
    except Exception as e:
        print(e)
    print("----save----")


In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import time
if __name__=='__main__':
    headers={'user-agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Mobile Safari/537.36',
         'accept-language': 'zh_TW'}
    proxy=['60.251.62.27','47.243.196.231','54.37.160.93','199.19.226.12','160.16.233.90','36.234.46.7','23.254.250.94']
    domain='https://www.klook.com'
    start=4
    NO=0
    times=0
    data=[]; tmp1={}; tmp2={}
    ALLdic2={};alldict2=[]
    yid={"ID":times}
    request('product',start,NO)
    show(data)
    